# Classifying Food Poisoning Message using Unsupervised Learning

The objective of this notebook is to address the reported cases of food poisoning among patrons who dined at a restaurant in suburban Banglore.



## Methodology

The objective of this notebook is achieved by using an unsupervised learning technique known as `clustering`. 

Clustering is an unsupervised learning method is a method in which we draw references from datasets consisting of input data without labeled responses. Generally, it is used as a process to find meaningful structure, explanatory underlying processes, generative features, and groupings inherent in a set of examples.



This notebook will be using